# Derivation and Abuse of the EKF Method for Real-World Slammage
Kalman Filters are used for recursively estimating the state of a linear system with noise or uncertainty. Extended Kalman Filters (EKF) allow this technique to be applied to nonlinear systems. This facilitates an EKF-based solution to the SLAM problem.


### Part 0 - Preface
This guide page assumes a relative degree of comfort with probability and linear algebra. None of the math here is super nasty but there are a lot of steps so being comfortable with matrix operations (i.e. multiplications, what is a Transposition and why is it used, what is a Jacobian and why is it used), and clearly understanding what an expectation of a distribution is will go a long way. I will briefly summarize these now for the precocious uninitiated:

1. WTF is a Transposition and why is it used?: $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, \quad \mathbf{x}^T = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}$ Now we can multiply these two vectors together. We can also do the same thing with matrices.

2. WTF is a Jacobian and why is it used?: Special matrix we use to get the the first-order derivative of a vectorial function: $J_f = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \cdots & \frac{\partial f_1(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n(x)}{\partial x_1} & \frac{\partial f_n(x)}{\partial x_2} & \cdots & \frac{\partial f_n(x)}{\partial x_n}
\end{bmatrix}, \quad J_ff(\cdot) = \nabla f(\cdot)$

3. WTF is an expectation and why is it used?: Most likely value of a Random Variable. Think of it as the weighted average across all outcomes, with each outcome multiplied by it's respective probability: $\mathbb{E}[X] = \int_{-\infty}^{\infty} x \, p(x) \, dx$ or $\mathbb{E}[X] = \sum_{x} x \, p(x)$ depending on if the RV is continuous or discrete.

I also want to make the disclaimer that while robotics is used as the context for explaining why some of the math below works out the way it does, it's not the only way. EKF can be used for other things as well, but this whole piece is centered around explaining it in the context of SLAM, and so robotics becomes the chosen medium.
### Part 1 - The Derivation

#### 1.0 - Preliminaries

For now we will just focus on the robot's state, $x_k$. We will define the state vector distribution as some function of the previous state distribution, $f(x_{k-1})$, with zero-mean gaussian noise, $w_{k-1}$, added to it to account for jump uncertainty (for example, uncertainty in the path-planning algo). We will define our observation vector distribution, $z_k$, as some function of the current state, $f(x_k)$, with zero-mean gaussian noise, $v_{k}$, added to it to account for observation uncertainty (sensor noise). This manifests the following two equations:


>$$
>(1) \quad x_k = f(x_{k-1}) + w_{k-1} 
>$$
>$$(2) \quad z_k = f(x_k) + v_{k-1} 
>$$

We define the initial state, $x_0$, as a random vector with known mean $\mu_0 = \mathbb{E}[x_0]$ and covariance $P_0 = \mathbb{E}\left[(x_0 - \mu_0)(x_0 - \mu_0)^T\right]$ 

From here, we are going to make a few assertions:

1. Noise vector distributions have a zero-valued mean (I explained the intuition behind this in the previous part): $\quad\mathbb{E}[w_k] = 0, \quad\mathbb{E}[v_k] = 0$  

2. The two types of noise vectors never correlate with each other: $\quad\mathbb{E}[w_k v_k^T j] = 0 \quad \forall k, j$  

3. Neither noise vector has any correlation with the start point: $\quad\mathbb{E}[w_k x_0^T] = 0 \quad \mathbb{E}[v_k x_0^T] = 0 \quad \forall k$  

4. Neither noise vector has any correlation whatsoever with any of its' predecessors or successors: $\quad\mathbb{E}[w_k w_j^T] = 0, \quad \mathbb{E}[v_k v_j^T] = 0 \quad  \forall k \neq j$

5. Vector variance (some sources use the term covariance but I think this is silly, because covariance with oneself is just variance) is represented with the following two matrices: $\quad\mathbb{E}[w_k w_k^T] = Q_k, \quad \mathbb{E}[v_k v_k^T] = R_k$


We also make the assumption that functions $f(\cdot)$ and $h(\cdot)$ as well as their first-order derivatives are continuous on the given domain.

To summarize, we have made a laundry-list of assertions to ensure that no Brimless Yankee activities will break what we do next. Dimensionality may be implictly known by now but it is nonetheless summarized below:


$$
x_k, \quad n \times 1 \quad\text{-- State vector at time step} \space k \newline
$$
$$
w_k, \quad n \times 1 \quad\text{-- Process noise vector} \newline
$$
$$
z_k, \quad m \times 1 \quad\text{-- Observation vector at time step} \space k \newline
$$
$$
v_k, \quad m \times 1 \quad\text{-- Measurement noise vector} \newline
$$
$$
f(\cdot), \quad n \times 1 \quad\text{-- Process nonlinear vector function (known)} \newline
$$
$$
h(\cdot), \quad m \times 1 \quad\text{-- Observation nonlinear vector function (known)} \newline
$$
$$
Q_k, \quad n \times n \quad\text{-- Process noise covariance matrix} \newline
$$
$$
R_k, \quad m \times m \quad\text{-- Measurement noise covariance matrix} \newline
$$
$$
K_k, \quad m \times m \quad\text{-- Kalman gain matrix} \newline
$$

> [!NOTE]
>
>In most (all?) real-world use-cases as far as SLAM is concerned, $n$ should always occupy a value between 1 and 3 inclusive. However, in order not to anger our high-dimensional overlords (the same mf's responsible for various lighthearted glitch-in-the-matrix activities such as always having a 747 headed to Anchorage flying directly over any rural BC town during any given observation period, Quantum Tunneling of the car keys, or running into some very specific individual at very random times/places in a way and at a frequency that probabilistically just doesn't make sense), we are keeping $n$ abstractly defined. As you may imagine, $m$ is abstractly defined because it depends on the specifics of how the sensor suite collects data.

#### 1.1 - Model Forecast Step (Time Update)

This is analogous to the "Time Update" part of our generalized SLAM algorithm. In the beginning, the only information have is the mean, $\mu_0$. We use this to obtain our initial optimal estimate, $x_0^a$ and variance, $P_0$ in the following manner:

>$$
>(3) \quad x_0^a = \mu_0 = \mathbb{E}[x_0]
>$$
>$$
>(4) \quad P_0 = \mathbb{E}[(x_0 - x_0^a)(x_0 - x_0^a)^T]
>$$

This is intuitive for our initial optimal estimate because we would expect it to be at the starting point that is by definition the most likely. We don't condition on any observation for this step because we operate under the assumption that $\mu_0$ is obtained either through prior contextual knowledge, or an initial guess. Of course, that intial knowledge/guess could be obtained (at least in part) through sensor readings, but then it wouldn't really make sense to condition it on those same sensor readings thereafter. For peace of mind, we will assume that our overlords gift us this information via high-dimensional carrier pigeon, and any sensor readings taken at this location thereafter would only add impurity to the divine estimate. We will not pain ourselves with the specifics of how it was obtained any further.


We are only gifted one high-dimensional carrier piegon, however. This means that, assuming our sensor data is on average more accurate than our odometry data, the following states' ideal estimates should incorporate conditioning from the sensor data in order to be ideal.

>$$
>(5) \quad x_{k-1}^a \equiv \mathbb{E}[x_{k-1} | Z_{k-1}]
>$$

Now, for reasons that are not yet obvious, let's say we want to get Taylor's Version (Taylor Aproximation) of $f(x_{k-1})$. We can do that as follows:

$$f(x_{k-1}) \equiv f(x_{k-1}^a) + J_f(x_{k-1} - x_{k-1}^a) + H.O.T.$$

Where the Jacobian, $J_f$, is defined as:

$$
J_f = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \cdots & \frac{\partial f_1(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n(x)}{\partial x_1} & \frac{\partial f_n(x)}{\partial x_2} & \cdots & \frac{\partial f_n(x)}{\partial x_n}
\end{bmatrix}
$$

In the spirit of Taylor, we simplify for the masses:

>$$(6) \quad f(x_{k-1}) \approx f(x_{k-1}^a) + J_fe_{k-1}$$

Where $e_{k-1} \equiv x_{k-1} - x_{k-1}^a$ and the higher-order terms are dropped because they have negligible effect on the result. Now we have a convenient way of estimating $f(x_{k-1})$ knowing only $f(\cdot)$, $x_{k-1}^a$, and $x_{k-1}$. As you may expect, at this point this is completely useless because we dont know $x_{k-1}$, and we also wouldn't know $x_{k-1}^a$ unless $k-1 = 0$. As you may expect in addition to the previously expected, all following math in this document is dedicated towards ultimately rectifying this.

Now what we want to do is figure out how to get the most accurate possible 'time update' value for $x_k$, which we denote as $x_k^f$. To do this we consider the Markov (Bumbling) process from part 0:
1. Move to a new state $x_k$ as a function of the previous state $f(x_{k-1})$
2. Make an observation $z_k$ that is probably more accurate than our odometry data.

From this, it's relatively intuitive to see that the best way we could get an estimate for $x_k^f$ would be to take the expectation of $f(x_{k-1})$ conditioned on distribution $Z_{k-1}$. Doing so yields the following:

$$
x_k^f = \mathbb{E}[f(x_{k-1}) | Z_{k-1}] \approx \mathbb{E}[(f(x_{k-1}^a) + J_f(x_{k-1}^a)e_{k-1}) | Z_{k-1}]
$$
$$
= f(x_{k-1}^a) + J_f(x_{k-1}^a)\mathbb{E}[e_{k-1} | Z_{k-1}]
$$
>$$
>(7) \quad \therefore x_k^f \approx f(x_{k-1}^a)
>$$

>[!NOTE] Of course, it's a lot harder to actually PROVE that that really is the IDEAL estimate (in the least squares sense) but for now just take my word for it.

The expectation of $f(x_{k-1}^a) | Z_{k-1}$ is $f(x_{k-1}^a)$ because $x_{k-1}^a$ is itself an expectation conditioned on $Z_{k-1}$.

The expectation of $e_{k-1} | Z_{k-1}$ is zero because we expect, on average, a delta of zero between our ideal (and unbiased) estimate and our actual value. As with $x_{k-1}^a$, this delta will already have the observation conditioning baked in. 

>[!NOTE]
>
>Of course, sensor bias does exist in the real world but for now we will assume that either it doesn't, or that we can compensate for it in a deterministic manner.

Now we have a decent enough approximation for $x_k^f$, which is itself a function of our best possible approximation for $x_{k-1}$, but this causes a new problem to appear: $x_k$ is a distribution, not a single number like $x_k^f$. In addition to this, it's not like we really knew what our $x_{k-1}$ was in the first place because it itself was also a distribution, we just took our best guess, $x_{k-1}^a$. 


We now have a formula for $x_k^f$ which lets us expand on two terms that we will make use of later. The first of these terms is forecast error, $e_k$:
>$$
>(8) \quad e_k^f \equiv x_k - x_k^f
>$$
$$
= \mathbf{f}(\mathbf{x}_{k-1}) + \mathbf{w}_{k-1} - \mathbf{f}(\mathbf{x}_{k-1}^a) 
$$
$$
\approx \mathbf{J}_f(\mathbf{x}_{k-1}^a) \mathbf{e}_{k-1} + \mathbf{w}_{k-1}
$$


The second of these terms is our forecast error variance, $P_k^f$:
$$
P_k^f \equiv \mathbb{E}[e_k^f(e_k^f)^T]
$$
$$
= J_f(x_{k-1}^a)\mathbb{E}[e_{k-1}e_{k-1}^T]J_f^T(x_{k-1}^a) + E[w_{k-1}w_{k-1}^T]
$$
>$$
>(9) \quad \therefore P_k^f = J_f(x_{k-1}^a)P_{k-1}J_f^T(x_{k-1}^a) + Q_{k-1}
>$$


The next thing we have to do is take $x_k$ and condition on our observation $Z_k$ in order to obtain the ideal estimate used for the next time step of the process: $x_k^a$



#### 1.2 - Data Assimilation Step (Measurement Update)
We now seek to condition our forecast value, $x_k^f$, in order to obtain our ideal unbiased estimate (ideal in the least squares sense), $x_k^a$, which estimates $x_k$ and is used to drive the upcoming forecast in the next cycle of our Markov process. To do this, we assert that $x_k^a$ is some linear combination of $x_k^f$ and observation $z_k$ (not to be confused with distribution $Z_k$, which we don't really know as it depends on the nonlinear distribution of $x_k$), let:

>$$
>(10) \quad x_k^a = a + K_kz_k
>$$

where $a$ contains $x_k^f$ and some other stuff.


From the notion that our estimate is unbiased, we can assert that the expected error is zero:

>$$(11) \quad 0 = \mathbb{E}[x_k - x_k^a|Z_k]$$

$$ = \mathbb{E}[(x_k^f + e_k^f) - (a + K_kh(x_k) + K_kv_k)|Z_k]$$

$$ = x_k^f - a - K_k\mathbb{E}[h(x_k)|Z_k]$$

$$\therefore a = x_k^f - K_k\mathbb{E}[h(x_k)|Z_k]$$ 

After substituting (11):

>$$(12) \quad x_k^a = x_k^f + K_k(z_k - \mathbb{E}[h(x_k)|Z_k])$$

Now we once again take Taylor's version to rewrite $h(x_k)$ in terms of $h(x_k^f)$:

$$h(x_k) \equiv h(x_k^f) + J_h(x_k^f)(x_k - x_k^f) + H.O.T$$

>$$(13) \quad \therefore h(x_k) \approx h(x_k^f) + J_h(x_k^f)(e_k^f)$$

Which allows us to approximate our expectation:

>$$(14) \quad \mathbb{E}[h(x_k)|Z_k] \approx h(x_k^f)$$

This is because $x_k^f$ is itself an estimate and is conditionally independant of the observation. The conditional independance is enforced by the Markov property which states that $x_k$ can only depend on the previous state, $x_{k-1}$, and the control signal input, $u_k$, as per part 0. In other words, it must strictly be a function of the previous state, $f(x_{k-1})$, which drives the control signal. We use our best estimate, $x_{k-1}^a$, to approximate our most likely jump, $x_k^f$, and then add noise, $w_{k-1}$, to model the process uncertaintly resulting from our fallible estimate. At the end of the day, however, the Markov property must still be satisfied. The expected error for $e_k^f$ is zero because the noise added to $x_k$ has a mean value of zero and the mean delta between $x_{k-1}^a$ and $x_{k-1}$ is also zero given that our observations are unbiased, thus: $\quad \mathbb{E}[e_k^f] = \mathbb{E}[x_k - x_k^f] \approx \mathbb{E}[f(x_{k-1}) + w_{k-1} - f(x_{k-1}^a)] = 0$.

>[!NOTE] The more intuitive way to think about this: Any new observation $z_k$ is made after the forecast state $x_k^f$ is already obtained, so $z_k$ is not able to give any new information on $x_k^f$ because $x_k^f$ is already locked in.

Substituting (15) into (13):

>$$(15) \quad x_k^a \approx x_k^f + K_k(z_k - h(x_k^f))$$

Now we can find all error signals and their covariance matrices. This involves relatively hideous amounts of algebra that I have neither the time nor attention span to write here so I'll spare you your time and attention as well. You can find the math starting at equation 24 in [this here document](../resources/tutorialEKF.pdf). I will now briefly summarize the remaining formulas:

>$$(16) \quad e_k \equiv x_k - x_k^a \approx (I - K_kJ_h(x_k^f))J_f(x_{k-1}^a)e_{k-1} + (I - K_kJ_h(x_k^f))w_{k-1} - K_kv_k$$

Through substitution of (15), where $I$ is the identity matrix.

Through substitution of (17): 
>$$(17) \quad P_k \equiv E[e_ke_k^T] = P_k^f - K_kJ_h(x_k^f)P_k^f - P_k^fJ_h^T(x_k^f)K_k^T + K_kJ_h(x_k^f)P_k^fJ_h^T(x_k^f)K_k^T + K_kR_kK_k^T$$

This gives us our posterior error variance, which relates to the expected deviation between our real value $x_k$, and our estimate, $x_k^a$.

Since (10), we have been using matrix $K_k$ to transform our observation vector in order to make our math work out. This is the Kalman Gain Matrix, and it is obtained by asserting that $P_k$'s trace's rate of change with respect to $K_k$ is zero and then solving for $K_k$. Very briefly, trace is the sum of the diagonal elements in a matrix. In a square matrix, like what $P_k$ by definition (18) must be, trace is also the sum of it's Eigenvalues. This means given that $P_k$ reperesents expected uncertainty in our estimates, the Eigenvalues represent the total uncertainty across all state variables in the system. We don't like uncertainty, and thus we try to minimize it. Geometrically, you can picture this as the covariance matrix defining an error ellipsoid in n-dimensional space, wherein each Eigenvalue represents the intersection of the elipsoid's surface across any particular axis. If we minimize the overall sum of our Eigenvalues, we reduce the overall amount of error (between $x_k$ and $x_k^a$) in our system. This is desired because once again, we seek to minimize uncertainty and thus error. Given how this matrix was obtained (expected squares of error vector components), it's easy to see how all entries in it, including the Eigenvalues, will be positive; this by definition means our matrix is **positive semidefinite**. Because of this, we will never have a global maximum, only a global minimum. Therefore, we can use the following formula to obtain $K_k$ that results in the least amount of error in our algorithm:


$$0 = \frac{\partial tr(P_k)}{\partial K_k} = -(J_h(x_k^f)P_k^f)^T - P_k^fJ_h^T(x_k^f) + 2K_kJ_h(x_k^f)P_k^fJ_h^T(x_k^f) + 2K_kR_k$$


>$$(18) \therefore K_k = P_k^fJ_h^T(x_k^f)(J_h(x_k^f)P_k^fJ_h^T(x_k^f) + R_k)^{-1} $$


Substituting this into (18) yields the following:

>$$(19) P_k = (I - K_kJ_h(x_k^f))P_k^f$$


#### 1.3 - Summary So Far

We are now tired and a bit delirious so lets summarize what we just spent a bunch of time doing. We know the ideal estimate for the robot's starting state, and we have deterministic functions that give us the forecasted jump state as a function of the previous state, as well as the observation vector as a function of the current state.

$$x_0^a = u_0 \quad\text{with error variance}\quad P_0$$
$$ x_k = f(x_{k-1}) + w_{k-1} $$
$$ z_k = f(x_k) + v_{k-1}$$

$x_k$ has a nonlinear distribution because the functions $f(\cdot)$ and $h(\cdot)$ are necessarily nonlinear functions. They are nonlinear functions because they depend on what the robot is trying to do at any given moment. This makes exact propagation of the state distribution computationally infeasible in most cases.

The whole point of the EKF is to allow approximation of these nonlinear state distributions without ever knowing the preceeding state distribution, $x_{k-1}$, directly. This is done through clever application of linear algebra and probability theory to recursively estimate and linearly approximate at each step of the process. To do this, we seek help from our friend, Taylor.

For the time update, we can estimate the distribution of $f(x_{k-1})$ for any given $k-1$ by taking the first-order Taylor approximation (Taylor's Version) centered around the best estimate of what our current state is, $x_{k-1}^a$ from measurement update at time period $k-1$. 


$$f(x_{k-1}) \approx f(x_{k-1}^a) + J_f(x_{k-1}^a)(x_{k-1} - x_{k-1}^a)$$


Through this we get an estimate for the forecast state:

$$x_k^f \approx f(x_{k-1}^a)$$

Now we can get an approximation of the ideal state estimate, $x_k^a$, as a linear combination of the forecast state estimate and our observation vector, $z_k$, which is the vector obtained from our sensor readings. 

$$x_k^a \approx x_k^f + K_k(z_k - h(x_k^f))$$

We need a matrix operating on $(z_k - h(x_k^f))$ for this linear approximation to work. We name this matrix the Kalman Gain Matrix and denote it as $K_k$. From a functional perspective, you can think of the Kalman Gain as the mechanism that that adapts to counteract compounding uncertainties resulting from successive Taylor approximations of each state. 

$$ K_k = P_k^fJ_h^T(x_k^f)(J_h(x_k^f)P_k^fJ_h^T(x_k^f) + R_k)^{-1} $$


The Kalman Gain is itself dependant on the measurement noise variance (which is known) as well as the forecast error variance, $P_k^f$

$$P_k^f = J_f(x_{k-1}^a)P_{k-1}J_f^T(x_{k-1}^a) + Q_{k-1}$$


This variance is itself dependant on the process noise variance (which is a known because we made it up), and the previous measurement update error variance, $P_{k-1}$.
Measurement update error variance was obtained through linear algebra witchraft briefly alluded to in (19) and elaborated on in equations (25) and (28) within [that there document](../resources/tutorialEKF.pdf). It takes on its final form below:

$$P_k = (I - K_kJ_h(x_k^f))P_k^f$$


$x_k^f$ estimates recurse back to base case $f(x_0^a) = f(u_0)$ whereas $P_k^f$ estimates recurse back to base case $J_f(x_0^a)P_0J_f^T(x_0) + Q_0$, wherein all variables are known. We know $f(x_0^a)$ because we know $f(\cdot)$ and we know $x_0^a$. Once again, the reason we can't just linearly map input deltas to output deltas is because $f(\cdot)$ is inherently nonlinear. $z_k$ is a known at all states because it is an environment reading that we ourselves obtain.

This allows us to recursively solve for $x_k^a$ at any time period $k$. This value is our best guess of where the robot is at that particular time i.e. if you asked the robot for its location, it would say that it was at $x_k^a$.

Throughout each time step $k$ of this recursive process, noise is added back into the system via variance matrices $Q_k$ and $R_k$ in order to allow for the Kalman Gain to adapt to system uncertainty (which is driven largely by the nonlinear behaviour of $f(\cdot)$ and $h(\cdot)$), thus producing a more accurate estimate for $x_k^a$ as a result. This allows the compounding errors via successive linear approximations of nonlinear functions to be minimized. In other words, we accept that we can't directly know how our functions behave through recursive linear approximation, because the functions themselves are nonlinear, so we linearly adapt our output estimations as a function of the output uncertainty around that particular point. Over time, there is less error compounding by following this method because while our estimates may never be equal to the exact location like they could have been before, we also have an overall lower chance of making a big miss, and as such the overall rate of error is lower.

However, as it stands right now this system will still ultimately diverge if landmark locations feeding the observations are not accurately known. We need more information to drive convergence, and we do that through an EKF-Based abuse of the generic SLAM algorithm detailed in [Part 0](../0-CoreAlgo/0-CoreAlgo.ipynb).


### Part 2 - The Abuse
Now we abuse our freshly derived EKF method in order to solve our SLAM problem. Thankfully, most of the work has already been done above so this will go really quickly:

#### 2.1 Quik Maffs

We first define our ideal estimates as follows:

> (Uno) $$ \begin{bmatrix} x_k^a \\ m_k^a \end{bmatrix} = \mathbb{E}\begin{bmatrix} x_k^a | Z_{0:k} \\ m_k^a | Z_{0:k} \end{bmatrix}$$


Next we define our Joint Covariance:

> [Dos] $$ P_{k} = \begin{bmatrix} P_{xx} P_{xm}\\ P^T_{xm} P_{mm} \end{bmatrix}_{k|k} \\ = \mathbb{E}\begin{bmatrix}\begin{pmatrix}x_k - x_k^a \\ m_k - m_k^a\end{pmatrix}\begin{pmatrix}x_k - x_k^a \\ m_k - m_k^a\end{pmatrix}^T | Z_{0:k}\end{bmatrix} $$


Now we basically do the exact same thing as in part 1, but also with the map vector as well:

> {Tres} $$ \begin{bmatrix} x_k^a \\ m_k^a \end{bmatrix} =  \begin{bmatrix} x_k^f \\ m_{k-1}^a \end{bmatrix} + K_k[z_k - h(x_k^f, m_{k-1}^a)] $$

$K_k$ is computed the exact same way as before, just substitute in our new $P_k$. $f(\cdot)$ and $h(\cdot)$ are now functions of the map inputs too and thus their Jacobians will be as well. $m_k^a$ recurses back to some starting map state $m_0^a$, which reflects our initial understanding of the map. It's clearly not perfect, and it doesn't need to be, but it gives us a base case on which to recursively build upon.

Alternatively, we can simplify our math via circular argument: 

> <El Quatro (mucho loco)> $$ \begin{bmatrix} x_k^a \\ m_k^a \end{bmatrix} = \begin{bmatrix} x_k^a \\ m_k^a \end{bmatrix} $$


#### 2.2 Convergence

Convergence of the system is achieved through monotonic convergence of landmark covariance submatrix $P_{mm}$'s determinent towards zero. This effectively results in all landmark-pair sub-matrices' covariances monotonically approaching zero, or more specifically, whatever the final level of system uncertainty is as a function of physical constraints. Individual landmark variances monotonically approach a lower bound as the transformation applied to them via $P_{mm}$ becomes negligible, this is due to the fact that the impact of new observations is minimized as the system converges (spring network analogy). In the spirit of zero, the intuition behind this is elaborated on in [Part Zero](../README.md).


This is really nice, because we don't have to adapt the method very much in order to drive convergence. It's significantly less nice when considering computational effort though, because all landmarks and thus the joint covariance matrix must be updated every time a new observation is made. 


#### 2.3 Further Reading for Additional Self-Imposed Suffering (Enrichment)
